In [21]:
import sys

sys.path.append("..")

import numpy as np

import tensorflow as tf

from utils import tf_config, preprocess_data, viz_utils

from layers import fft

In [2]:
tf_config.set_seed(seed=42)#02 20
tf_config.setup_tensorflow(device="GPU", memory_limit=1500)

path_labels="../../metrics/10_synth_01_style_prior_bayesian/"
view="fmri"

2022-04-01 09:32:44.115784: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-01 09:32:44.116528: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-01 09:32:44.168121: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-01 09:32:44.168864: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:0a:00.0 name: GeForce RTX 2080 SUPER computeCapability: 7.5
coreClock: 1.845GHz coreCount: 48 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 462.00GiB/s
2022-04-01 09:32:44.168894: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-04-01 09:32:44.172286: I tensorflow/stream_executor/platform/def

In [5]:
X_view = np.load(path_labels+"views.npy")
y_true = np.load(path_labels+"y_true.npy")
y_pred = np.load(path_labels+"y_pred.npy")

### Downsample

In [35]:
downsample_shape=(14,14,7)
dct=fft.DCT3D(*X_view.shape[1:-1])
idct=fft.iDCT3D(*downsample_shape)

X_view_downsampled=np.empty((0,)+downsample_shape+(1,), dtype="float32")

for i in range(X_view.shape[0]):
    X_view_downsampled=np.append(X_view_downsampled,np.expand_dims(idct(dct(X_view[i:i+1,:,:,:,0])[:,:downsample_shape[0],:downsample_shape[1],:downsample_shape[2]]).numpy(), axis=-1),axis=0)

### Build arff format

In [65]:
string_file_view_ground_truth=""
string_file_view_pred=""
for individual in range(X_view_downsampled.shape[0]):
    string_file_view_ground_truth+=str(int(y_true[individual]))+","
    string_file_view_pred+=str(int(y_pred[individual]))+","
    for value in np.reshape(X_view_downsampled[individual], (downsample_shape[0]*downsample_shape[1]*downsample_shape[2],)):
        string_file_view_ground_truth+="{:0.9f}".format(float(value))+","
        string_file_view_pred+="{:0.9f}".format(float(value))+","
    string_file_view_ground_truth=string_file_view_ground_truth[:-1]+"\n"
    string_file_view_pred=string_file_view_pred[:-1]+"\n"

In [67]:
with open("./view_ground_truth.arff", "w") as f:
    f.write(string_file_view_ground_truth)
    f.close()

In [ ]:
with open("./view_pred.arff", "w") as f:
    f.write(string_file_view_pred)
    f.close()